## Forest Health Threshold Update - Results
* Mason Bindl mbindl@trpa.gov
* Andrew McClary amcclary@trpa.gov

### Setup

In [ ]:
from sqlalchemy.engine import URL, create_engine

# default workspace
workspace     = r'C:\GIS\Python\Scripts\Demographics'
arcpy.env.workspace = "C:\\GIS\\Scratch.gdb"

# path to database connection files
filePath = r"C:\\GIS\\DB_CONNECT"
# filePath = "F:\\GIS\\GIS_DATA"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")

# # sql database connection with pyodbc
# conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')
# conn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff')
# setup sql connection
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url    = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
conn              = create_engine(connection_url)


In [ ]:
import os
import csv
from datetime import datetime
import getpass
import pyodbc
import xlrd
import arcpy
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer, FeatureLayerCollection
import pickle
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
py.init_notebook_mode()
# plotly settings
init_notebook_mode(connected=False)

# pandas settings
pd.options.plotting.backend = "plotly"
# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
memory = "memory/"

# default workspace
workspace           = r"\\TRPA-FS01\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct"
arcpy.env.workspace = r"\\TRPA-FS01\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb"

# path to database connection files
filePath = "C:\\GIS\\DB_CONNECT"
# filePath = "F:\\GIS\\GIS_DATA"

# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")

# sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')
conn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff')

# set style variables
template = 'plotly_white'
font     = 'Calibri'

# colors for tables
headerColor = '#6680a8'
rowColor = 'white'
lastrowColor = '#eeeeee'

### General Stats

In [ ]:
# climate classes, acres, descriptions>
dfClimateClass = pd.read_csv(os.path.join(workspace, "ClimateClasses_TahoeValues.csv"))
dfClimateClass


In [ ]:
dfClimateClass.info()

In [ ]:
# climate classes, acres, descriptions>
dfClimateClass = pd.read_csv(os.path.join(workspace, "ClimateClasses_TahoeValues.csv"))
df = dfClimateClass
# drop fields to add up row totals
df.drop(columns=['OID_','Count'], inplace=True)

# format values
df['Acres']   = df['Acres'].map(u"{:,.0f}".format)
df['FractalIndex10thPercentile']   = df['FractalIndex10thPercentile'].map(u"{:,.2f}".format)
df['FractalIndex90thPercentile']   = df['FractalIndex90thPercentile'].map(u"{:,.2f}".format)

# format table column names
headers = ['<b>Climate Class</b>', '<b>Stand Density 10th Percentile</b>',
           '<b>Stand Density 90th Percentile</b>', '<b>Large Tree Density 10th Percentile</b>',
           '<b>Large Tree Density 90th Percentile</b>',  '<b>Fractal Index 10th Percentile</b>',
           '<b>Fractal Index 90th Percentile</b>','<b>Acres</b>']

# setup table figure
fig = go.Figure(data=[go.Table(
    columnorder = [1,2,3,4,5,6,7,8],
    columnwidth = [20,10,10,10,10,10,10,20],
    header=dict(values=headers,
                fill_color='#6680a8',
                align='center',
                line_color='darkslategray',
                font=dict(color='white', size=14),
                height=20),
    cells=dict(values=[df['Value'] , df['StandDensity10thPercentile'], 
                       df['StandDensity90thPercentile'],df['LargeTreeDensity10thPercentile'], 
                       df['LargeTreeDensity90thPercentile'],df['FractalIndex10thPercentile'], 
                       df['FractalIndex90thPercentile'],df['Acres']],
               fill_color = [[rowColor]],
               line_color='darkslategray',
               font=dict(size=14),
               align=['left','center'],
               height=30))
])

# update style and margins
fig.update_layout(
    font_family=font,
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()

# save to HTML
fig.write_html(os.path.join(workspace, "ClimateClass_Table.html"))
# save PNG
fig.write_image(os.path.join(workspace, "CliamteClass_Table.png"))

## Stand Density

In [46]:
dfStandDensityClass = pd.read_csv(os.path.join(workspace, "StandDensity_TPA_Classified_30m_Tahoe.csv"))

In [47]:
df = dfStandDensityClass

In [ ]:
# stand density class bar chart
fig = px.bar(df, y="Acres", x="Category", 
             title="Stand Density Class",
            )
fig.update_traces(marker_color='#6680a8', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
# show figure
fig.show()
# save to HTML
fig.write_html(os.path.join(workspace, "StandDensityClass.html"))
# save to PNG
fig.write_image(os.path.join(workspace, "StandDensityClass.png"))

In [ ]:
df = dfStandDensityClass
# drop fields to add up row totals
df.drop(columns=['OID_','Value','Count'], inplace=True)

# format values
df['Acres']   = df['Acres'].map(u"{:,.0f}".format)

# format table column names
headers = ['<b>Class</b>','<b>Acres</b>']

# setup table figure
fig = go.Figure(data=[go.Table(
    columnorder = [1,2],
    columnwidth = [20,20],
    header=dict(values=headers,
                fill_color='#6680a8',
                align='center',
                line_color='darkslategray',
                font=dict(color='white', size=14),
                height=20),
    cells=dict(values=[df['Category'] , df['Acres']],
               fill_color = [[rowColor]*5+[lastrowColor]],
               line_color='darkslategray',
               font=dict(size=14),
               align=['left','center'],
               height=30))
])

# update style and margins
fig.update_layout(
    font_family=font,
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()

# save to HTML
fig.write_html(os.path.join(workspace, "StandDensityClass_Table.html"))
# save PNG
fig.write_image(os.path.join(workspace, "StandDensityClass_Table.png"))

## Large Tree Density

In [ ]:
dfLargeTreeDensityClass = pd.read_csv(os.path.join(workspace, "LargeTreeDensity_Class_10th90thPercentile.csv"))
dfLargeTreeDensityClass

In [ ]:
df = dfLargeTreeDensityClass
# drop fields to add up row totals
df.drop(columns=['OID_','Value','Count'], inplace=True)

# format values
df['Acres']   = df['Acres'].map(u"{:,.0f}".format)

# format table column names
headers = ['<b>Class</b>','<b>Acres</b>']

# setup table figure
fig = go.Figure(data=[go.Table(
    columnorder = [1,2],
    columnwidth = [20,20],
    header=dict(values=headers,
                fill_color='#6680a8',
                align='center',
                line_color='darkslategray',
                font=dict(color='white', size=14),
                height=20),
    cells=dict(values=[df['Category'] , df['Acres']],
               fill_color = [[rowColor]*5+[lastrowColor]],
               line_color='darkslategray',
               font=dict(size=14),
               align=['left','center'],
               height=30))
])

# update style and margins
fig.update_layout(
    font_family=font,
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()

# save to HTML
fig.write_html(os.path.join(workspace, "LargeTreeDensityClass_Table.html"))
# save PNG
fig.write_image(os.path.join(workspace, "LargeTreeDensityClass_Table.png"))

In [ ]:
df

In [ ]:

df = dfLargeTreeDensityClass

# stand density class bar chart
fig = px.bar(df, y="Acres", x="Category", 
             title="Large Tree Density Class",
            )
fig.update_traces(marker_color='#6680a8', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
# show figure
fig.show()
# save to HTML
fig.write_html(os.path.join(workspace, "LargeTreeDensityClass.html"))
# save to PNG
fig.write_image(os.path.join(workspace, "LargeTreeDensityClass.png"))

In [ ]:
# using 20th and 80th percentile cutoffs
dfLargeTreeDensityClass2080 = pd.read_csv(os.path.join(workspace, "LargeTreeDensity_Class_20th80thPercentile.csv"))
df = dfLargeTreeDensityClass2080

# stand density class bar chart
fig = px.bar(df, y="Acres", x="Category", 
             title="Large Tree Density Class",
            )
fig.update_traces(marker_color='#6680a8', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
# show figure
fig.show()
# save to HTML
fig.write_html(os.path.join(workspace, "LargeTreeDensityClass2080.html"))
# save to PNG
fig.write_image(os.path.join(workspace, "LargeTreeDensityClass2080.png"))

In [ ]:
df = dfLargeTreeDensityClass2080
# drop fields to add up row totals
df.drop(columns=['OID_','Value','Count'], inplace=True)

# format values
df['Acres']   = df['Acres'].map(u"{:,.0f}".format)

# format table column names
headers = ['<b>Class</b>','<b>Acres</b>']

# setup table figure
fig = go.Figure(data=[go.Table(
    columnorder = [1,2],
    columnwidth = [20,20],
    header=dict(values=headers,
                fill_color='#6680a8',
                align='center',
                line_color='darkslategray',
                font=dict(color='white', size=14),
                height=20),
    cells=dict(values=[df['Category'] , df['Acres']],
               fill_color = [[rowColor]*5+[lastrowColor]],
               line_color='darkslategray',
               font=dict(size=14),
               align=['left','center'],
               height=30))
])

# update style and margins
fig.update_layout(
    font_family=font,
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()

# save to HTML
fig.write_html(os.path.join(workspace, "LargeTreeDensityClass_Table2080.html"))
# save PNG
fig.write_image(os.path.join(workspace, "LargeTreeDensityClass_Table2080.png"))

## Horizontal Heterogeneity

In [ ]:
dfFractalIndexClass = pd.read_csv()

## Seral Stage and Canopy Cover

In [ ]:
dfSeral = 
dfCanopy=


## Functional Fire

In [ ]:
dfFire = pd.read_csv()
